In [ ]:
from os import path
import pandas as pd
import pickle
from tqdm import tqdm
import json
import pandas as pd
import requests

In [ ]:
def pocket_url(endpoint):
    return "https://getpocket.com/v3/" + endpoint

In [ ]:
def scrape_and_tag_articles(csv_path: str, ):

    # 'cm_pocket_export_23-11-2020.csv'
    data=pd.read_csv(args.pocket_csv_input, index_col=0)
    print("Input data Info: \n", data.info())

    #when scraping all articles
    solution = data

    if args.causes_only:
        solution_index=[]
        for index,row in data.iterrows():
            tags=row['tags']
            if 'cause'in tags:
                solution_index.append(index)
            elif 'causes' in tags:
                solution_index.append(index)
            elif 'Cause' in tags:
                solution_index.append(index)
            elif 'Causes' in tags:
                solution_index.append(index)
            elif 'caused' in tags:
                solution_index.append(index)
            elif 'causing' in tags:
                solution_index.append(index)

        solution=data.loc[solution_index]

    fb_string='https://www.facebook.com/'
    fb_solution_index=[]
    for index,row in solution.iterrows():
        if fb_string in row['given_url']:
            fb_solution_index.append(index)

    solution_wo_fb=solution.drop(index=fb_solution_index)

    # XXX: Need better naming
    solution_wo_fb.to_csv(path.join(args.output_dir, 'all_pocket.csv'))

    articles=solution_wo_fb['resolved_url'].values.tolist()

    token = args.diffbot_token
    URL   = args.diffbot_url

    #data_to_store=[]
    #articles=solution_wo_fb['resolved_url'].values.tolist()
    #arricles_processed=[]

    for x in tqdm(range(2206, 3084)):
        PARAMS = {'token':token,'url':articles[x]}
        r = requests.get(url = URL, params = PARAMS)
        data = r.json()
        data_to_store.append(data)
        arricles_processed.append(articles[x])

    pickle.dump( data_to_store, open( path.join(args.output_dir, "all_pocket.p"), "wb" ) )

In [ ]:
def process_pocket_pickle():
    #"all_pocket_diffbot_extract.p"
    diffbot_extract_file = args.diffbot_extract_file
    diffbot_extract      = open(diffbot_extract_file, "rb")
    data                 = pickle.load(diffbot_extract)

    # data[0]['objects'][0].keys()
    # 'date', 'sentiment', 'images', 'author', 'estimatedDate', 'publisherRegion', 'icon', 'diffbotUri', 'siteName',
    # 'type', 'title', 'tags', 'publisherCountry', 'humanLanguage', 'pageUrl', 'html', 'text'

    # "all_pocket_tag_annotations.csv"
    tags_file_path = args.tags_file_path

    #read all pocket tag annotations csv file into pandas dataframe
    tag_annotations = pd.read_csv(tags_file_path, sep = "\t")

    annotations_df = tag_annotations[["pageURL","tags"]]
    annotations_df.tags = annotations_df["tags"].apply(lambda x: x.strip('][').replace("'","").split(', '))

    #resolve duplicates by merging them.
    annotations_df = annotations_df.groupby(['pageURL']).agg(sum).reset_index()
    unique_tag_annotations = annotations_df.copy()
    unique_tag_annotations.tags = annotations_df['tags'].apply(lambda x: list(set(x)))

    text_list = []

    for article in data:

        try:
            article_json = article['objects']

            if (article_json[0]["type"]):
                doc_type = article_json[0]["type"]
                if (doc_type == "article"):
                    text     = article_json[0]["text"]
                    url      = article_json[0]["pageUrl"]
                    title    = article_json[0]["title"]
                    siteName = article_json[0]["siteName"]

                    diffbot_tags = get_if_exists(article_json[0], "tags")
                    unique_id    = get_if_exists(article_json[0], "diffbotUri")
                    date         = get_if_exists(article_json[0], "date")
                    author       = get_if_exists(article_json[0], "author")

                    if (url in unique_tag_annotations['pageURL'].tolist()):
                        pocket_tags = unique_tag_annotations.loc[unique_tag_annotations['pageURL'] == url , "tags"].tolist()[0]

                    if (pocket_tags):
                        items = {"text" : text, "title" : title, "diffbot_tags": diffbot_tags, \
                                 "date": date, "author": author, "siteName": siteName, \
                                 "unique_id": unique_id, "url": url, "doc_type": doc_type,\
                                 "pocket_tags": ", ".join(pocket_tags)}
                    else:
                        items = {"text" : text, "title" : title, "diffbot_tags": diffbot_tags, \
                                 "date": date, "author": author, "siteName": siteName, \
                                 "unique_id": unique_id, "url": url, "doc_type": doc_type}

                    # example tag: effects
                    if (args.tag_only):
                        if(pocket_tags and args.tag_only in pocket_tags):
                            text_list.append(items)

        except Exception as e:
            print("Error: ", e)
            print("Article: ", article, "\n")

    with open(args.output_file_path, 'w') as f:
        for item in text_list:
            f.write(json.dumps(item) + "\n")